In [22]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Activation, Flatten, Dense,Reshape
from tensorflow.keras.layers import Conv2D, AveragePooling2D, BatchNormalization, GlobalAveragePooling1D, GlobalAvgPool2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import activations
import time
import math

In [2]:
from tensorflow.keras.datasets import cifar10
(img_train, label_train), (img_test, label_test) = cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [3]:
x_train = img_train.astype('float32') / 255
x_test = img_test.astype('float32') / 255

In [4]:
y_train = to_categorical(label_train)
y_test = to_categorical(label_test)

In [ ]:
model = load_model('drive/My Drive/Do_an-NCKH/model_Conv2D.h5')

In [5]:
def polyfit(px):
  return 0.1524*(px**2) + 0.5*px + 0.409

In [ ]:
model_poly = load_model('/content/drive/My Drive/Do_an-NCKH/model_poly.h5', custom_objects={'polyfit': polyfit})

#Client

In [33]:
x = x_test[0:2]
x.shape

(2, 32, 32, 3)

In [34]:
stride, kernel, padding = 1, 3, 0

In [35]:
leghtX = x.shape[0]
heightX = x.shape[1]
weightX = x.shape[2]
deptX = x.shape[3]

In [36]:
H = math.floor((heightX - kernel + 2*padding) / stride) + 1
W = math.floor((weightX - kernel + 2*padding) / stride) + 1
L = kernel*kernel*deptX

In [37]:
X = np.zeros((leghtX, H, W, L + 1, 2, 2))
k = np.array([[np.random.rand(),np.random.rand()],
              [np.random.rand(),np.random.rand()]])
kd = np.linalg.inv(k)

In [38]:
for xi in range(leghtX):
  for hi in range(0, heightX, stride):
    for wi in range(0, weightX, stride):
      x_ = x[xi,hi:hi+kernel,wi:wi+kernel,:].reshape(-1)
      for i_ in np.arange(L):
        X_ = np.array([[x_[i_], np.random.rand()],
                       [np.random.rand(), np.random.rand()]])
        X[xi, hi, wi, i_] = k.dot(X_)
        # X[xi, hi, wi, i_] = 0.5 * X_
        if i_ == L - 1:
          X__ = np.array([[1, 1],
                          [1, 1]])
          X[xi, hi, wi, L] = k.dot(X__)
      if wi + kernel == weightX:
        break
    if hi + kernel == heightX:
      break

#Server

In [39]:
wx = np.zeros((X.shape[0],30,30,96,2,2))

In [40]:
model_poly = model_relu #Tên là relu nhưng model e vẫn để hàm là poly ạ

In [41]:
w0, b0 = model_relu.layers[0].get_weights()
model_ = Sequential()
for layer in model_poly.layers[1:]:
  model_.add(layer)
model_.build(input_shape = model_poly.layers[0].output_shape)

#Xem1

In [42]:
listw = []
for d in np.arange(w0.shape[-1]):
  w_ = w0[:,:,:,d].reshape(-1)
  listw.append(np.append(w_,b0[d]))
w = np.array(listw)

In [ ]:
w.shape

(96, 28)

In [43]:
for xi in np.arange(X.shape[0]):
  for hi in np.arange(X.shape[1]):
    for wi in np.arange(X.shape[2]):
      for w_ in np.arange(w.shape[0]):
        sumR = 0
        for ri in np.arange(X.shape[3]):
          r = w[w_,ri]*X[xi, hi, wi, ri]
          sumR += r
        wx[xi, hi, wi, w_] = sumR

In [44]:
x00 = model_.predict(wx[:,:,:,:,0,0] )
x01 = model_.predict(wx[:,:,:,:,0,1] )
x10 = model_.predict(wx[:,:,:,:,1,0] )
x11 = model_.predict(wx[:,:,:,:,1,1] )
xr = np.zeros((2,10,2,2))
for i in range(2):
  for j in range(10):
    xr_ = np.array([[x00[i,j], x01[i,j]],
                   [x10[i,j], x11[i,j]]])
    xr__ = kd.dot(xr_)
    # xr__ = xr_ 
    xr[i,j] = xr__
lst = []
for ii in range(2):
  lst.append(xr[ii,:,0,0])
print(lst)

[array([-0.0314985 , -0.03070592, -0.03311249, -0.03259278, -0.03457322,
       -0.03081967, -0.03419587, -0.0326086 , -0.03153162, -0.03289231]), array([-0.03096774, -0.03045508, -0.03337809, -0.03265729, -0.03508717,
       -0.03072467, -0.03478696, -0.03278015, -0.03101845, -0.03267529])]


In [45]:
model_relu.predict(x)

array([[0.10056635, 0.09869021, 0.10036007, 0.10147744, 0.10327712,
        0.09644118, 0.100028  , 0.09881978, 0.09983113, 0.10050869],
       [0.10110447, 0.09895356, 0.10007829, 0.10144784, 0.10275545,
        0.09654878, 0.09941307, 0.09861941, 0.10035541, 0.1007238 ]],
      dtype=float32)

In [ ]:
y_uncrypt = model_poly.predict(x)
lst_uncrypt = []
for jj in range(y_uncrypt.shape[0]):
  lst_uncrypt.append(y_uncrypt[jj].argmax()) 
print(lst_uncrypt)

Thuật toán: Đối với CNN: Client phải có Stride, padding, kernel của model, tính toán output. Rồi gửi về Server ở ví dụ này X có shape (5,30,30,27,2,2). Phía Server tính toán sắp xếp lại shape của weight ở đây là (32,27), rồi sau đó tính toán như code trên để output giống như input lúc nhận. Sau đó cho vào model dự đoán từng giá trị của ma trận (2,2). Gửi về client. Client giải mã bằng K^(-1)

In [ ]:
p = 0.09
0.12*(p**2) + 0.5*p + 0.409

0.454972

In [6]:
img_size = 32

In [ ]:
model_poly = Sequential()
img_input = Input(shape=(img_size,img_size,3))
 
model_poly.add(Conv2D(96, (3,3), padding='valid', input_shape = img_input.shape[1:]))
# model_poly.add(Activation(polyfit))
# model_poly.add(AveragePooling2D(pool_size=(2,2), padding='valid'))
model_poly.add(BatchNormalization())
model_poly.add(Conv2D(96, (3,3),padding='valid'))
model_poly.add(BatchNormalization())
model_poly.add(Conv2D(96, (3,3),padding='valid'))
model_poly.add(Activation(polyfit))
model_poly.add(BatchNormalization())
 
model_poly.add(Conv2D(192, (3,3), padding='valid', input_shape = img_input.shape[1:]))
model_poly.add(BatchNormalization())
model_poly.add(Conv2D(192, (3,3), padding='valid', input_shape = img_input.shape[1:]))
model_poly.add(BatchNormalization())
model_poly.add(Activation(polyfit))
model_poly.add(Conv2D(192, (3,3), padding='valid', input_shape = img_input.shape[1:]))
model_poly.add(BatchNormalization())
model_poly.add(Conv2D(192, (3,3), padding='valid', input_shape = img_input.shape[1:]))
model_poly.add(BatchNormalization())
model_poly.add(Conv2D(192, (3,3), padding='valid', input_shape = img_input.shape[1:]))
model_poly.add(BatchNormalization())
model_poly.add(Activation(polyfit))
 
model_poly.add(Flatten())
model_poly.add(Dense(10))
# model_poly.add(GlobalAveragePooling2D())
model_poly.add(Activation(activations.softmax))

In [30]:
model_relu = Sequential()
img_input = Input(shape=(img_size,img_size,3))
 
model_relu.add(Conv2D(96, (3,3), padding='valid', input_shape = img_input.shape[1:]))
model_relu.add(Activation(polyfit))
model_relu.add(Conv2D(96, (3,3)))
model_relu.add(Activation(polyfit))
model_relu.add(Conv2D(96, (3,3), strides=2))
model_relu.add(Activation(polyfit))

model_relu.add(Conv2D(192, (3,3)))
model_relu.add(Activation(polyfit))
model_relu.add(Conv2D(192, (3,3)))
model_relu.add(Activation(polyfit))
model_relu.add(Conv2D(192, (3,3), strides=2))
model_relu.add(Activation(polyfit))

model_relu.add(Conv2D(192,(3,3)))
model_relu.add(Activation(polyfit))
model_relu.add(Conv2D(192,(1,1)))

model_relu.add(Activation(polyfit))
model_relu.add(Conv2D(10, (1,1)))
model_relu.add(GlobalAveragePooling2D())

model_relu.add(Activation(activations.softmax))

In [31]:
sgdx = SGD(lr=0.005, decay =1e-6, momentum=0.9, nesterov=True)
ntrain = 60
otp = Adam(learning_rate=1e-6)

In [ ]:
s = time.time()
model_poly.compile(optimizer=otp,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
H_new = model_poly.fit(x_train, y_train, epochs=15)
print('time: ',time.time() - s)

In [ ]:
s = time.time()
model_relu.compile(optimizer=otp,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
H_new = model_relu.fit(x_train, y_train,
                    epochs=10)
print('time: ',time.time() - s)

In [ ]:
 
model_poly.evaluate(x_test, y_test)

313/313 [==============================] - 8s 25ms/step - loss: 2.3679 - accuracy: 0.7017


[2.3679356575012207, 0.70169997215271]

#Test